# Load UBISc Data

Loads bills, sponsorships, and votes from Openstates, and exports to Bigquery.

In [1]:
!pip install pyopenstates

  Using cached pyopenstates-1.1.0-py2.py3-none-any.whl


In [2]:
import pyopenstates
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
pyopenstates.set_api_key('b6e279fd-5dd2-4bdf-b5f5-0520748f8f11')

List search queries.

eic is omitted as it primarily [yields](https://openstates.org/api/v1/bills/?q=eic&apikey=b6e279fd-5dd2-4bdf-b5f5-0520748f8f11) Spanish-language bills from Puerto Rico.

TODO: Create mechanism to add specific bills.

In [4]:
queries = ['earned income credit', 'earned income tax credit', 'eitc',
           'child tax credit', 'ctc', 'cdctc', 
           'basic income', 'negative income tax',
           # For Hawaii's HCR89 bill mandating research into UBI.
           'Basic Economic Security',
           # For California's carbon dividend proposal, SB775 (17 false positives).
           'market-based compliance mechanisms',
           # Colorado's EITC.
           'Colorado Working Families Economic Opportunity Act']
bill_fields = ['id', 'state', 'session', 'bill_id', 'title', 'created_at']
all_fields = list(bill_fields)
all_fields.append('sponsors')

In [5]:
bills = pd.DataFrame()
sponsorships = pd.DataFrame()
for i in queries:
    print i
    tmp_bills_json = pyopenstates.search_bills(q=i,  fields=all_fields)
    tmp_bills = pd.DataFrame(tmp_bills_json)
    tmp_bills['query'] = i
    # Ignore errors since some queries may not return bills with sponsors.
    bills = pd.concat([bills, tmp_bills.drop('sponsors', axis=1, errors='ignore')])
    tmp_sponsorships = json_normalize(tmp_bills_json, 'sponsors', 'id')
    sponsorships = pd.concat([sponsorships, tmp_sponsorships])

earned income credit
earned income tax credit
eitc
child tax credit
ctc
cdctc
basic income
negative income tax
Basic Economic Security
market-based compliance mechanisms
Colorado Working Families Economic Opportunity Act


Why the extra fields?

In [6]:
sponsorships = sponsorships[['id', 'leg_id', 'name', 'type']]

In [7]:
sponsorships[:5]

,id,leg_id,name,type
0,MAB00024531,MAL000451,Marjorie C. Decker,primary
1,MAB00024531,MAL000499,Steven Ultrino,cosponsor
2,MAB00024531,MAL000456,Jay R. Kaufman,cosponsor
3,MAB00024531,MAL000557,Leonard Mirra,cosponsor
4,MAB00024531,MAL000452,Jack Lewis,cosponsor


In [8]:
sponsorships.shape

(3340, 4)

In [9]:
bills.shape

(506, 7)

In [10]:
bills.tail(3)

,bill_id,created_at,id,query,session,state,title
16,AB 1906,2012-02-24 02:06:18,CAB00008585,market-based compliance mechanisms,20112012,ca,California Global Warming Solutions Act of 200...
17,SB 864,2011-03-24 20:45:17,CAB00006137,market-based compliance mechanisms,20112012,ca,Emissions of greenhouse gases: market-based co...
0,SB 13-001,2013-01-10 00:10:21,COB00001394,Colorado Working Families Economic Opportunity...,2013A,co,Colorado Working Families Economic Opportunity...


Deduplicate bills with multiple search queries.

In [11]:
dedup = pd.DataFrame({'queries' : bills.groupby(bill_fields)['query'].apply(lambda x: "{%s}" % ', '.join(x))}).reset_index()

In [12]:
pd.DataFrame(dedup.groupby('queries').size()).reset_index().sort_values(0, ascending=False)

,queries,0
8,{earned income tax credit},324
6,{earned income credit},93
4,{ctc},23
10,{market-based compliance mechanisms},18
9,{eitc},17
5,"{earned income credit, earned income tax credit}",8
3,{child tax credit},7
0,{Basic Economic Security},2
7,"{earned income tax credit, eitc}",2
1,{Colorado Working Families Economic Opportunit...,1


Strip timestamp from `created_at` for better exporting to Sheets (via Bigquery).

In [13]:
dedup['created_at'] = dedup['created_at'].dt.strftime('%Y-%m-%d')

Add openstates URL

In [14]:
dedup['openstates_url'] = 'https://openstates.org/' + dedup['state'] + '/bills/' + dedup['session'] + '/' + dedup['bill_id']

Export to BigQuery.

In [15]:
from datalab.context import Context
import google.datalab.storage as storage
import datalab.bigquery as bq

TODO: Use this function once the [magics SO question](https://stackoverflow.com/q/46722938/1840471) is resolved.

In [22]:
def write_bq(df, bucket_name, bigquery_table_name, bigquery_dataset_name):
    full_bucket_name = Context.default().project_id + '-' + bucket_name
    bucket_path = 'gs://' + full_bucket_name
    bucket_object = bucket_path + '/' + bucket_name + '.txt'

    # Define storage bucket
    bucket = storage.Bucket(full_bucket_name)

    # Create storage bucket if it does not exist
    if not bucket.exists():
        bucket.create()

    # Define BigQuery dataset and table
    dataset = bq.Dataset(bigquery_dataset_name)
    table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

    # Create BigQuery dataset
    if not dataset.exists():
        dataset.create()

    # Create or overwrite the existing table if it exists
    table_schema = bq.Schema.from_dataframe(df)
    table.create(schema = table_schema, overwrite = True)

    # Write the DataFrame to GCS (Google Cloud Storage)
    obj = bucket.object(bucket_object)
    obj.write_stream(df)
    #%storage write --variable df --object $bucket_object

    # Write the DataFrame to a BigQuery table
    table.insert_data(df)

In [23]:
# Params (would be function args).
df = dedup
bucket_name = 'openstates-bills'
bigquery_table_name = 'OpenstatesBills' 
bigquery_dataset_name = 'ubisc'

In [24]:
write_bq(df, bucket_name, bigquery_table_name, bigquery_dataset_name)

TypeError: write_stream() takes exactly 3 arguments (2 given)

In [44]:
# write_bq(dedup, 'openstates-bills', 'OpenstatesBills', 'ubisc')
# write_bq(sponsorships, 'openstates-sponsorships', 'OpenstatesSponsorships', 'ubisc')

In [45]:
full_bucket_name = Context.default().project_id + '-' + bucket_name
bucket_path = 'gs://' + full_bucket_name
bucket_object = bucket_path + '/' + bucket_name + '.txt'

# Define storage bucket
bucket = storage.Bucket(full_bucket_name)

# Create storage bucket if it does not exist
if not bucket.exists():
    bucket.create()

# Define BigQuery dataset and table
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists
table_schema = bq.Schema.from_dataframe(df)
table.create(schema = table_schema, overwrite = True)

# Write the DataFrame to GCS (Google Cloud Storage)
%storage write --variable df --object $bucket_object

# Write the DataFrame to a BigQuery table
table.insert_data(df)

id,state,session,bill_id,title,created_at,queries,openstates_url
MIB00000103,mi,2011-2012,SB 103,Income tax; credit; earned income tax credit; eliminate. Amends sec. 272 of 1967 PA 281 (MCL 206.272).,2011-04-07,{earned income tax credit},https://openstates.org/mi/bills/2011-2012/SB 103
WIB00003444,wi,2011 Regular Session,AB 637,"disregarding a taxpayer's election to include another in its combined group, disallowing certain carry-forward amounts for combined reporting purposes, repealing the income and franchise tax credit for qualified production activities income, repealing the changes made to the earned income tax credit in 2011 Wisconsin Act 32, restoring indexing provisions to the homestead tax credit, eliminating the individual income tax exclusion for long-term capital gains other than for farm assets, computing the estate tax based on 2002 federal law, and creating a new individual income tax upper bracket.",2012-02-28,{earned income tax credit},https://openstates.org/wi/bills/2011 Regular Session/AB 637
TXB00003552,tx,81,HB 956,Relating to the provision by the Texas Education Agency of federal earned income tax credit educational materials to school districts for distribution to students' parents.,2010-06-16,{earned income tax credit},https://openstates.org/tx/bills/81/HB 956
KYB00002712,ky,2013RS,SB 54,AN ACT relating to the Kentucky Board of Emergency Medical Services. Amend KRS 311A.015 to attach the Kentucky Board of Emergency Medical Services to the Public Protection Cabinet for administrative purposes; amend KRS 311A.020 to require all employees assigned to the board to be transferred from the KCTCS system to the KRS Chapter 18A personnel system; require new employees to be hired under provisions of KRS Chapter 18A; require the transfer of all moneys appropriated to and collected by the board to the Public Protection Cabinet; amend KRS 12.020 and 12.252 to conform.,2013-01-12,{ctc},https://openstates.org/ky/bills/2013RS/SB 54
NJB00021908,nj,215,S 2104,Increases the State earned income tax credit from 20 percent to 25 percent of the federal earned income tax credit over two taxable years.,2012-06-22,{earned income tax credit},https://openstates.org/nj/bills/215/S 2104
NYB00098118,ny,2017-2018,A 2108,"Provides earned income tax credit to youth workers, increases standard deduction for individuals 18 to 24, and provides for deduction of student loan interest",2017-01-23,{earned income tax credit},https://openstates.org/ny/bills/2017-2018/A 2108
PRB00016364,pr,2017-2020,PC 732,"Para enmendar la Sección 1052.01 de la Ley 1-2011, según emendada, conocida como “Código de Rentas Internas para un Nuevo Puerto Rico”, a los fines de restituir el crédito por trabajo (“earned income tax credit”); aumentar los límites de ingresos, con el propósito de fomentar el trabajo e impulsar nuestra economía; y para otros fines relacionados.",2017-02-07,{earned income tax credit},https://openstates.org/pr/bills/2017-2020/PC 732
IAB00003041,ia,2013-2014,SF 295,"A bill for an act relating to state and local finances by establishing a business property tax credit for commercial, industrial, and railway property, establishing and modifying property assessment limitations, providing for commercial and industrial property tax replacement payments, providing for the classification of multiresidential property, modifying provisions for the taxation of telecommunications company property, providing for the study of the taxation of telecommunications company property, providing a taxpayers trust fund tax credit, modifying provisions relating to the property assessment appeal board, modifying the amount of the earned income tax credit, making appropriations, providing penalties, and including effective date, implementation, retroactive applicability, and other applicability provisions. (Formerly SSB 1135.) Various effective dates; see sections 22, 30, 38, 45, and 64 of bill.",2013-03-05,{earned income tax credit},https://openstates.org/ia/bills/20

In [46]:
# Params (would be function args).
df = sponsorships
bucket_name = 'openstates-sponsorships'
bigquery_table_name = 'OpenstatesSponsorships' 
bigquery_dataset_name = 'ubisc'

In [47]:
full_bucket_name = Context.default().project_id + '-' + bucket_name
bucket_path = 'gs://' + full_bucket_name
bucket_object = bucket_path + '/' + bucket_name + '.txt'

# Define storage bucket
bucket = storage.Bucket(full_bucket_name)

# Create storage bucket if it does not exist
if not bucket.exists():
    bucket.create()

# Define BigQuery dataset and table
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists
table_schema = bq.Schema.from_dataframe(df)
table.create(schema = table_schema, overwrite = True)

# Write the DataFrame to GCS (Google Cloud Storage)
%storage write --variable df --object $bucket_object

# Write the DataFrame to a BigQuery table
table.insert_data(df)

Failed to process HTTP response.


id,leg_id,name,type
ILB00059444,ILL000725,LaShawn K. Ford,cosponsor
MDB00003246,MDL000202,Brian E. Frosh,cosponsor
ILB00097006,ILL000863,Deb Conroy,cosponsor
NCB00002878,NCL000172,Weiss,cosponsor
MDB00017455,MDL000605,A. Washington,cosponsor
NCB00005652,NCL000181,Keever,cosponsor
MIB00002821,MIL000064,Jim Townsend,cosponsor
NYB00023405,NYL000114,Gottfried,cosponsor
ILB00073100,ILL000076,John D. Cavaletto,cosponsor
CAB00017592,CAL000385,Maienschein,cosponsor


## Extract sponsors and votes